#Crawling Berita

## Crawling BeritaJateng

In [1]:
import requests
from bs4 import BeautifulSoup

def scrap_beritajateng(link, kategori):
    category = kategori
    data = []
    base_url = link
    page = 1  # Mulai dari halaman pertama atau tidak tergantung halaman
    while True:  # Loop sampai tidak ada lagi konten yang dimuat
        url = f'{base_url}page/{page}/'
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            blok = soup.find('div', class_='gmr-box-content hentry gmr-archive clearfix')
            articles = blok.find_all('div', class_='item-article')
            for article in articles:
                article_link = article.find('a')['href']
                title = article.find('h2').text.strip()
                content_response = requests.get(article_link)
                if content_response.status_code == 200:
                    content_soup = BeautifulSoup(content_response.content, "html.parser")
                    article_content = content_soup.find('div', class_='entry-content entry-content-single')
                    if article_content:
                        content_text = ""
                        for paragraph in article_content.find_all(['p', 'h2', 'h3', 'h4', 'h5']):
                            content_text += paragraph.get_text(strip=True).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                        data.append([title, content_text, category])

            # Cek apakah ada tombol "View More" atau tidak
            view_more_button = soup.find('button', {'class': 'view-more-button heading-text'})
            if view_more_button:
                # Jika ada, lakukan permintaan untuk memuat lebih banyak konten
                page += 1  # Ganti halaman jika ada tombol "View More"
            else:
                break  # Jika tidak ada tombol "View More", keluar dari loop
        else:
            break  # Keluar dari loop jika permintaan tidak berhasil
    return data


In [6]:
politik = scrap_beritajateng('https://beritajateng.net/category/politik/','Politik')
olahraga = scrap_beritajateng('https://beritajateng.net/category/olahraga/','Olahraga')

In [17]:

kriminal = scrap_beritajateng('https://beritajateng.net/category/hukum-kriminal/', 'Kriminal')

In [18]:
import pandas as pd

hasil1 = pd.DataFrame(politik, columns=['Judul', 'Isi', 'Kategori'])
hasil2 = pd.DataFrame(olahraga, columns=['Judul', 'Isi', 'Kategori'])
hasil3 = pd.DataFrame(kriminal, columns=['Judul', 'Isi', 'Kategori'])

##Scraping Berita BantenNews

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
import requests
from bs4 import BeautifulSoup
import time

def scrap_beritajatim(link, hal, kategori):
    category = kategori
    data = []
    base_url = link
    for page in range(1, hal + 1):
        url = f'{base_url}page/{page}/'
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            blok = soup.find('div', class_='td-ss-main-content')
            articles = blok.find_all('div', class_='td-block-span6')
            for article in articles:
                link = article.find('a')['href']
                title = article.find('h3').text.strip()
                content_response = requests.get(link)
                if content_response.status_code == 200:
                    content_soup = BeautifulSoup(content_response.content, "html.parser")
                    _class_content = content_soup.find('div', class_='td-ss-main-content')
                    # tanggal = content_soup.find('time')  # Mengambil elemen tanggal
                    if _class_content:
                        content_text = ""
                        for paragraph in _class_content.find_all(['p']):
                            content_text += paragraph.get_text(strip=True).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                        # tanggal_publish = tanggal.get_text(strip=True)  # Mendapatkan tanggal publikasi
                        data.append([title, content_text, category])

                time.sleep(0.5)  # Tambahkan waktu tunggu 0.5 detik sebelum permintaan data berikutnya

    return data


In [8]:
politikbanten = scrap_beritajatim('https://www.bantennews.co.id/category/politik/',32,'Politik')

In [9]:
sport = scrap_beritajatim('https://www.bantennews.co.id/category/olahraga/',32,'Olahraga')

In [10]:
hukum = scrap_beritajatim('https://www.bantennews.co.id/category/hukum/',30,'Kriminal')

In [12]:
import pandas as pd

hasil4 = pd.DataFrame(politikbanten, columns=['Judul', 'Isi', 'Kategori'])
hasil5 = pd.DataFrame(sport, columns=['Judul', 'Isi', 'Kategori'])
hasil6 = pd.DataFrame(hukum, columns=['Judul', 'Isi', 'Kategori'])

In [19]:
combined_df = pd.concat([hasil1, hasil2, hasil3, hasil4, hasil5, hasil6]).reset_index(drop=True)
combined_df

,Judul,Isi,Kategori
0,Bawaslu Turunkan Atribut Kampanye Terpasang di...,SURAKARTA –Badan Pengawas Pemilu (Bawaslu) Kot...,Politik
1,Presiden Jokowi Makan Siang dengan Tiga Bakal ...,JAKARTA –Presiden Joko Widodo mengundang tiga ...,Politik
2,Prabowo Jelaskan Kenapa Gibran tak Hadir Dekla...,JAKARTA –Ketua Umum Partai Gerindra sekaligus ...,Politik
3,Penyelenggara Pemilu Diminta Profesional,"SEMARANG– Penjabat (Pj) Gubernur Jawa Tengah, ...",Politik
4,488 Warga Ajukan Pindah Pemilih di Banyumas,BOYOLALI –Komisi Pemilihan Umum (KPU) Kabupate...,Politik
...,...,...,...
1053,Polsek Kragilan Kerahkan Tangki Penyaluran Air...,SERANG– Krisis air bersih dampak fenomena El N...,Kriminal
1054,Polres Serang Bongkar Industri Rumahan Tembaka...,SERANG– Tim Satgas Satuan Reserse Narkoba (Sat...,Kriminal
1055,Propam Periksa Penggunaan Senjata Personel Pol...,SERANG– Personel Propam Polres Serang melakuka...,Kriminal
1056,Ahli Waris Lahan SDN Kuranji Serang Klaim Puny...,"SERANG– Sengketa lahan SDN Kuranji, Kota Seran...",Kriminal


In [20]:
df = combined_df.sample(frac=1).reset_index(drop=True)
df.to_csv('crawlingberitauas.csv', index=False)

In [21]:
df

,Judul,Isi,Kategori
0,Pendiri Koperasi di Kudus Gelapkan Dana Nasaba...,"SEMARANG, 10/10 (BeritaJateng.net) – Jajaran D...",Kriminal
1,Cak Imin Yakin Duet AMIN Daftar Capres-Cawapre...,"SERANG– Ketua Umum PKB, Muhaimin Iskandar alia...",Politik
2,Polda Banten Apel Gelar Pasukan Operasi Mantap...,SERANG– Situasi politik menjelang Pemilu 2024...,Politik
3,Erick Thohir Beberkan Tiga Alasan Indonesia Te...,JAKARTA– Ketua Umum Persatuan Sepak Bola Selur...,Olahraga
4,Propam Periksa Penggunaan Senjata Personel Pol...,SERANG– Personel Propam Polres Serang melakuka...,Kriminal
...,...,...,...
1053,"Diserang Netizen Indonesia, Akun Instagram Tim...",SERANG– Tim Nasional (Timnas) Thailand harus m...,Olahraga
1054,Ratusan Off-Roader Motor Trail ‘Ulin Bareng’ d...,LEBAK– Masih dalam rangka memeriahkan Hari Ula...,Olahraga
1055,"Terlibat Kasus Korupsi, Hakim Perintahkan 4 St...",SERANG– Hakim Pengadilan Negeri (PN) Serang me...,Kriminal
1056,Hasil Drawing Piala Dunia U-17 2023: Ini Dia L...,JAKARTA– Hasil drawing Piala Dunia U-17 2023 t...,Olahraga


In [22]:
pd.crosstab(index=df['Kategori'], columns='count')

col_0,count
Kategori,
Kriminal,357
Olahraga,351
Politik,350
